# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096907


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:03<00:51,  1.79s/it]

Rendering models:  10%|▉         | 3/31 [00:04<00:40,  1.45s/it]

Rendering models:  13%|█▎        | 4/31 [00:04<00:32,  1.22s/it]

Rendering models:  16%|█▌        | 5/31 [00:05<00:27,  1.06s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:26<00:02,  1.62it/s]

Rendering models:  90%|█████████ | 28/31 [00:27<00:02,  1.45it/s]

Rendering models:  94%|█████████▎| 29/31 [00:27<00:01,  1.54it/s]

Rendering models: 100%|██████████| 31/31 [00:28<00:00,  1.84it/s]

not-logged-in-673e227f920b24558cc1    0.000042
not-logged-in-673e227f920b24558cc1    0.000019
ethanth                               0.000035
ChelseaB98                            0.000139
not-logged-in-8cf2e6929490167ca6ab    0.000532
jnarayanbvg                           0.000055
RTitchenal                            0.000029
not-logged-in-a0d39065895c84f0fd2d    0.000589
salmi.a01                             0.000084
xStormmyy                             0.000747
AmeenDeen                             0.209249
TSM_Ginga                             0.000033
ArtisticFangirl24                     0.013128
iesfranciscoayala                     0.000048
JanEltner                             0.000045
crush202020                           0.000037
JulieHedglin                          0.000054
AlexYoung35                           0.000030
pangeli5                              0.000043
surfgimp                              0.000032
not-logged-in-906234f52dd4bb00491f    0.000028
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())